In [1]:
import numpy as np
import pandas as pd
# import plotly.express as px
# from autokeras import StructuredDataRegressor
from sklearn.model_selection import train_test_split
# from IPython.display import HTML
import os
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2022-09-05 22:06:18.118548: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [3]:
from ASC_ML.networkbuilding import multiple_model_gen_v3 as multiple
from ASC_ML.networkbuilding import dataframe_extractor as de
from ASC_ML.networkbuilding import model_generation as model_gen
from ASC_ML.networkbuilding import model_optimization as model_opt
from ASC_ML.networkbuilding import model_stacking

In [4]:
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

In [8]:
import dask.dataframe as dd

In [9]:
from ASC_ML.preprocessing import dataset_container as dc
from ASC_ML.preprocessing import data_cleaning as data_cl

In [10]:
df = dd.read_csv("/home/anish/Downloads/autokeras-regression-master/Data.csv")

In [11]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [12]:
target_col = "sales"
X = df.loc[:, df.columns != target_col]
y = df.loc[:, target_col]

In [13]:
data_cl_obj = data_cl.DataCleaning(["sales"], df)

In [14]:
data_cl_obj.col_info

{'TV': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 190},
 'radio': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 167},
 'newspaper': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 172},
 'sales': {'dtype': dtype('float64'),
  'is_label': True,
  'missing': 0.0,
  'cardinality': 121}}

In [15]:
data_cl_obj.dataset.get()[0].compute()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [16]:
data_cl_obj.dataset.get()[0]["sales"].compute()

0      22.1
1      10.4
2       9.3
3      18.5
4      12.9
       ... 
195     7.6
196     9.7
197    12.8
198    25.5
199    13.4
Name: sales, Length: 200, dtype: float64

In [17]:
directory_train = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"
directory_test = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/test.csv"
train_df = dd.read_csv(directory_train, assume_missing = True, sample_rows=1000)
test_df = dd.read_csv(directory_test, assume_missing = True, sample_rows=1000)

In [18]:
train_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal,208500.0
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal,181500.0
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal,223500.0
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal,250000.0


In [19]:
data_cl_obj = data_cl.DataCleaning(["SalePrice"], train_dataframe = train_df, test_dataframe = test_df)

In [20]:
data_cl_obj.col_info

{'Id': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 1460},
 'MSSubClass': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 15},
 'MSZoning': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 5},
 'LotFrontage': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.1773972602739726,
  'cardinality': 110},
 'LotArea': {'dtype': dtype('float64'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 1073},
 'Street': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 2},
 'Alley': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.9376712328767123,
  'cardinality': 2},
 'LotShape': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 4},
 'LandContour': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinality': 4},
 'Utilities': {'dtype': dtype('O'),
  'is_label': False,
  'missing': 0.0,
  'cardinal

In [21]:
data_cl_obj.encode()

In [22]:
data_cl_obj.dataset.get()[0].compute()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,0,65.0,8450.0,0,0,0,0,0,...,0.0,0,0,0,0.0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,0,80.0,9600.0,0,0,0,0,0,...,0.0,0,0,0,0.0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,0,68.0,11250.0,0,0,1,0,0,...,0.0,0,0,0,0.0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,0,60.0,9550.0,0,0,1,0,0,...,0.0,0,0,0,0.0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,0,84.0,14260.0,0,0,1,0,0,...,0.0,0,0,0,0.0,12.0,2008.0,0,0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,0,62.0,7917.0,0,0,0,0,0,...,0.0,0,0,0,0.0,8.0,2007.0,0,0,175000.0
1456,1457.0,20.0,0,85.0,13175.0,0,0,0,0,0,...,0.0,0,1,0,0.0,2.0,2010.0,0,0,210000.0
1457,1458.0,70.0,0,66.0,9042.0,0,0,0,0,0,...,0.0,0,3,1,2500.0,5.0,2010.0,0,0,266500.0
1458,1459.0,20.0,0,68.0,9717.0,0,0,0,0,0,...,0.0,0,0,0,0.0,4.0,2010.0,0,0,142125.0


In [23]:
data_cl_obj.dataset.get(types = ["train"])[0].compute()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,0,65.0,8450.0,0,0,0,0,0,...,0.0,0,0,0,0.0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,0,80.0,9600.0,0,0,0,0,0,...,0.0,0,0,0,0.0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,0,68.0,11250.0,0,0,1,0,0,...,0.0,0,0,0,0.0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,0,60.0,9550.0,0,0,1,0,0,...,0.0,0,0,0,0.0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,0,84.0,14260.0,0,0,1,0,0,...,0.0,0,0,0,0.0,12.0,2008.0,0,0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456.0,60.0,0,62.0,7917.0,0,0,0,0,0,...,0.0,0,0,0,0.0,8.0,2007.0,0,0,175000.0
1456,1457.0,20.0,0,85.0,13175.0,0,0,0,0,0,...,0.0,0,1,0,0.0,2.0,2010.0,0,0,210000.0
1457,1458.0,70.0,0,66.0,9042.0,0,0,0,0,0,...,0.0,0,3,1,2500.0,5.0,2010.0,0,0,266500.0
1458,1459.0,20.0,0,68.0,9717.0,0,0,0,0,0,...,0.0,0,0,0,0.0,4.0,2010.0,0,0,142125.0


In [24]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [25]:
# X = data_cl_obj.dataset.get(types = ["train"])[0].compute()
X = data_cl_obj.dataset.get(types = ["train"])[0]
X = X.dropna().copy()
Y = X["SalePrice"]

In [26]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,0,65.0,8450.0,0,0,0,0,0,...,0.0,0,0,0,0.0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,0,80.0,9600.0,0,0,0,0,0,...,0.0,0,0,0,0.0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,0,68.0,11250.0,0,0,1,0,0,...,0.0,0,0,0,0.0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,0,60.0,9550.0,0,0,1,0,0,...,0.0,0,0,0,0.0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,0,84.0,14260.0,0,0,1,0,0,...,0.0,0,0,0,0.0,12.0,2008.0,0,0,250000.0


In [27]:
type(X)

dask.dataframe.core.DataFrame

In [28]:
Y

Dask Series Structure:
npartitions=1
    float64
        ...
Name: SalePrice, dtype: float64
Dask Name: getitem, 4 tasks

In [29]:
# X = data_cl_obj.dataset.get(types = ["train"])[0].compute()
# Y = data_cl_obj.dataset.get(types = ["train"])[0]["SalePrice"].compute()
start = time.time()
model = RandomForestClassifier()
n = 50
rfe = RFE(estimator = model, n_features_to_select=n)
fit = rfe.fit(X, Y)
X = X[X.columns[fit.support_]]
X.head()
end = time.time()

In [30]:
X.head()

,Id,MSSubClass,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,Condition1,HouseStyle,OverallQual,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,Fence,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,60.0,65.0,8450.0,0,0,0,0,0,7.0,...,548.0,0.0,61.0,0.0,0,2.0,2008.0,0,0,208500.0
1,2.0,20.0,80.0,9600.0,0,1,1,1,1,6.0,...,460.0,298.0,0.0,0.0,0,5.0,2007.0,0,0,181500.0
2,3.0,60.0,68.0,11250.0,1,0,0,0,0,7.0,...,608.0,0.0,42.0,0.0,0,9.0,2008.0,0,0,223500.0
3,4.0,70.0,60.0,9550.0,1,2,2,0,0,7.0,...,642.0,0.0,35.0,272.0,0,2.0,2006.0,0,1,140000.0
4,5.0,60.0,84.0,14260.0,1,1,3,0,0,8.0,...,836.0,192.0,84.0,0.0,0,12.0,2008.0,0,0,250000.0


In [ ]:
start = time.time()
m = multiple.Multiple_Model_Gen_V3(X_train, y_train, X_test, y_test, 50, 64, input_shape = 58, 
                                   max_no_layers = 3, model_per_batch = 10, 
                                   save_dir = "/home/anish/ASC_ML_test_weights/")
m.get_model_confs()
m.get_best_models(save = False)
opt = model_opt.Model_Optimization(X_train, y_train, X_test, y_test, 200, m.evaluate_dict_list, save_dir = "/home/anish/ASC_ML_test_weights/candidate_models/")
opt.optimize_models(save=True)
stacked_models = model_stacking.Model_Stacking(X_train, y_train, X_test, y_test, opt.saved_paths, opt.model_confs, save_dir = "/home/anish/ASC_ML_test_weights/stacked_models/")
stacked_models.optimize_stacked_models()

end = time.time()
print("Time Taken : ", (end-start))

2022-09-05 21:43:58.983746: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10



 
 

loss  :  0.15193229913711548 , TEST :  0.2884425222873688
output_layer_dense_32_32_loss  :  0.01729872077703476 , TEST :  0.025728963315486908
output_layer_dense_32_64_loss  :  0.017637794837355614 , TEST :  0.028626935556530952
output_layer_dense_32_128_loss  :  0.018964704126119614 , TEST :  0.030719580128788948
output_layer_dense_32_256_loss  :  0.012988993898034096 , TEST :  0.03047638200223446
output_layer_dense_32_512_loss  :  0.013915164396166801 , TEST :  0.02726651541888714
output_layer_dense_32_1024_loss  :  0.015199643559753895 , TEST :  0.028119483962655067
output_layer_dense_64_32_loss  :  0.013662857003509998 , TEST :  0.030536554753780365
output_layer_dense_64_64_loss  :  0.014764813706278801 , TEST :  0.028499247506260872
output_layer_dense_64_128_loss  :  0.012919753789901733 , TEST :  0.0289174672216177
output_layer_dense_64_256_loss  :  0.014579840935766697 , TEST :  0.02955138124525547

 
 

loss  :  0.12320762872695923 , TEST :  0.2824208736419678
output_laye


 
 

loss  :  0.11152472347021103 , TEST :  0.2775840759277344
output_layer_dense_64_256_64_loss  :  0.01100969035178423 , TEST :  0.026385335251688957
output_layer_dense_64_256_128_loss  :  0.014908239245414734 , TEST :  0.029181422665715218
output_layer_dense_64_256_256_loss  :  0.01032148115336895 , TEST :  0.027852343395352364
output_layer_dense_64_256_512_loss  :  0.008775010704994202 , TEST :  0.028140373528003693
output_layer_dense_64_256_1024_loss  :  0.00843245442956686 , TEST :  0.02641390450298786
output_layer_dense_64_512_32_loss  :  0.0086958147585392 , TEST :  0.02588636614382267
output_layer_dense_64_512_64_loss  :  0.01133755687624216 , TEST :  0.02834792248904705
output_layer_dense_64_512_128_loss  :  0.010454471223056316 , TEST :  0.028671175241470337
output_layer_dense_64_512_256_loss  :  0.014613140374422073 , TEST :  0.02821396104991436
output_layer_dense_64_512_512_loss  :  0.012976858764886856 , TEST :  0.028491267934441566

 
 

loss  :  0.12901481986045837 , T


 
 

loss  :  0.12010987102985382 , TEST :  0.25313326716423035
output_layer_dense_1024_32_32_loss  :  0.011990358121693134 , TEST :  0.024156656116247177
output_layer_dense_1024_64_32_loss  :  0.01018105074763298 , TEST :  0.025737950578331947
output_layer_dense_1024_64_64_loss  :  0.018407661467790604 , TEST :  0.028628623113036156
output_layer_dense_1024_128_32_loss  :  0.011116303503513336 , TEST :  0.024163343012332916
output_layer_dense_1024_128_64_loss  :  0.010509009473025799 , TEST :  0.024594217538833618
output_layer_dense_1024_128_128_loss  :  0.014213274233043194 , TEST :  0.025483740493655205
output_layer_dense_1024_256_32_loss  :  0.010404925793409348 , TEST :  0.025417828932404518
output_layer_dense_1024_256_64_loss  :  0.012432903051376343 , TEST :  0.02496338076889515
output_layer_dense_1024_256_128_loss  :  0.009356893599033356 , TEST :  0.02432086691260338
output_layer_dense_1024_256_256_loss  :  0.011497467756271362 , TEST :  0.02566663920879364

 
 

loss  :  0.12

2022-09-05 21:47:35.705796: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


--------------------------------------------------------------------------------
Model Name : dense_1024_64

BEST HYPERPARAMETERS : BEST_LOSS : 0.02007736638188362, BEST_ACTIVATION : selu, BEST_INITIALIZER : LecunUniform, BEST_LEARINING_RATE : 5e-05, BEST_BATCHSIZE : 16

 
 

loss  :  0.020848426967859268 , TEST :  0.036818601191043854
--------------------------------------------------------------------------------
Model Name : dense_1024_512_128

